# 딥러닝 이해
 - 인공지능 : 기계가 사람의 행동을 모방하게 하는 기술
 - 기계학습 : 기계가 일일이 코드로 명시하지 않은 동작을 데이터로부터 학습하여 실행할 수 있도록 하는 알고리즘을 개발하는 연구 분야
 - 딥러닝 : 기계 학습의 한 분야인 인공신경망에 기반하여, 많은 양의 데이터를 학습해 뛰어난 성능을 이끌어내는 연구 분야
 
# 기계학습 vs 딥러닝
#### (고양이, 개) - 특징 추출기 - 특징 벡터 - 분류기 - (결과)
#### (고양이, 개) - 특징 추출기 & 분류기 - (결과)
  - 기존에는 분류기만 학습하기 때문에 한계가 있음
  - 그러나 딥러닝은 특징 추출기까지 확장되어 좋음
  
# 딥러닝으로 할 수 있는 것
  - 분류(Classification), 회귀(Regression), 물체 검출(Object Detection), 영상 분할(Image Segmentation), 영상 초해상도(Image Super Resolution), 예술적 창조물(Artistic Creation with GAN), 강화 학습(Reinforcement Learning)
  
# 딥러닝 구성 요소
### 학습 단계(Training Phase)
  1. 학습 데이터 셋(Training Dataset) : 학습 입력(Training Input) + 학습 정답(Training Groundtruth)
  2. 네트워크 구조(Network Architecture)
  3. 손실 함수(Loss Function)
  4. 알고리즘 최적화 기법(Algorithm Optimizer)
  .[다시 2로 돌아가서 반복]

### 테스트 단계(Test Phase)
  1. 테스트 입력(Test Input)
  2. 학습된 네트워크(Trained Network)
  3. 평가 지표(Evaluation Metric)

# 딥러닝의 역사
 1. 1943 : Eletronic Brain
 2. 1957 : Perceptron
 3. 1960 : ADALINE
 4. 1969 : XOR Problem
   - 1960 ~ 1969 : Golden Age
   - 1969 ~ 1986 : Dark Age("AI Winter")
 5. 1986 : Multi-layered Perceptron(Backpropagation)
 6. 1995 : SVM
 7. 2006 : Deep Neural Network(Pretraining)
 
# 딥러닝의 대중화 - Framework
 - 초창기
  - Caffe2 : C++와 리눅스 기반
  - MatConvNet : 윈도우, MATLAB 환경에 익숙한 연구원 대상
 - 최근
  - TensorFlow와 Pytorch
 - 클라우드 플랫폼 3강 구도
  - AWS, GCP(Google Cloud Platform), Microsoft Azure
 - HW
  - GTX 시리즈는 게이밍 용이지만, 딥러닝 연구에서 최고의 가성비를 내면서 연구의 대중화를 이끔
 - IaaS : 하드웨어만 제공 받음
 - PaaS : Pytorch 등만 깔아서
 - SaaS : 다 깔려 있음

In [2]:
import tensorflow as tf

In [19]:
# 네트워크 구조 정의
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [20]:
# 학습 루프 정의
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model, trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [21]:
# 테스트 루프 정의
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [22]:
# 데이터셋 가져오고, 정리하는
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [23]:
# 모델 생성, 네트워크 생성
model = MyModel()

In [24]:
# 손실함수, 최적화 알고리즘 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [25]:
# 알고리즘 평가지표 설정
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [26]:
# 알고리즘 학습, 테스트 루프
for epoch in range(5):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {|, Test Loss: {}, Test Accuracy {}'
    print(template.format(epoch,
                         train_loss.result(),
                         train_accuracy.result() * 100,
                         test_loss.result(),
                         test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



NameError: in converted code:

    <ipython-input-6-0b0e1a72cd81>:7 train_step  *
        gradients = tape.gradient(loss, model, trainable_variables)

    NameError: name 'trainable_variables' is not defined
